# Etapa dos Projetos no pipeline do Salic

**ISSUE:** [#28](https://github.com/lappis-unb/salic-ml/issues/28)

**Etapas de Projeto:** No Salic um projeto por estar em uma das seguintes etapas:

- Admissibilidade
- Capitação e Execução
- Prestação de Contas
  - Análise do Objeto
  - Análise Financeira


**Objetivo:** Identificar quantos projetos tem em cada etapa no Salic

## Prepararando os dados

**Recarregar automaticamente os módulos**

In [2]:
%load_ext autoreload
%autoreload 2

**Importando módulos python**

In [3]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from salicml.utils.dates import Dates
from salicml.models.projects import Projects

PROJECT_ROOT = os.path.abspath(os.path.join(os.pardir, os.pardir))
DATA_FOLDER = os.path.join(PROJECT_ROOT, 'data', 'raw')

**Carregando os dados da tabela de Projetos**

In [4]:
cols = ['IdPRONAC', 'DtProtocolo', 'DtInicioExecucao', 'DtFimExecucao', 'Situacao']
projects_csv = os.path.join(DATA_FOLDER, 'projetos.csv')

projects = Projects(projects_csv)

df_projects = projects.data_frame.filter(items = cols)
display(df_projects.head())

,IdPRONAC,DtProtocolo,DtInicioExecucao,DtFimExecucao,Situacao
0,1,2008-09-12 00:00:00,2010-01-01 00:00:00,2010-12-31 00:00:00,E63
1,2,2008-09-12 00:00:00,NaN,NaN,G43
2,3,2008-09-12 00:00:00,NaN,NaN,K00
3,4,2008-09-03 00:00:00,NaN,NaN,E25
4,5,2008-09-12 00:00:00,NaN,NaN,E36


## Data na qual esse notebook foi executado

In [5]:
current_date = time.strftime("%Y-%m-%d %H:%M:%S")

display(time.strftime("%d-%m-%Y %H:%M:%S"))

'29-05-2018 14:57:54'

## Período da análise

Serão analizados os projetos criados apartir de 2009

In [6]:
year = 2009

df_projects = df_projects[df_projects['DtProtocolo'] >= '{}-01-01'.format(year)]

## Situações que indicam que um projeto foi encerrado

In [7]:
end_situations = ['A09', 'A13', 'A14', 'A16', 'A17', 'A18', 'A20', 'A23', 'A24', 'A26', 'A40', 'A41',
                  'A42', 'C09', 'D18', 'E04', 'E09', 'E36', 'E47', 'E49', 'E63', 'E64', 'E65', 'G16', 'G25',
                  'G26', 'G29', 'G30', 'G56', 'K00', 'K01', 'K02', 'L01', 'L02', 'L03', 'L04', 'L05', 
                  'L06', 'L08', 'L09', 'L10', 'L11']

## Identificando projetos na etapa de Admissibilidade

### Preparando filtros

Para identificar se um projeto está na etapa de admissibilidade será verificado as seguintes informações

1. A data de inicio de execução de um projeto deve ser posterior a data atual.
2. O projeto não pode ter uma situação que indica que ele já foi finalizado.

**Filtro 1:** A data de inicio de execução de um projeto deve ser posterior a data atual.

In [8]:
filter_start_execution_bigger_than_currentdate = df_projects['DtInicioExecucao'] > current_date

**Filtro 2:** O projeto não pode ter uma situação que indica que ele já foi finalizado.

In [9]:
filter_not_have_end_situation = ~df_projects['Situacao'].isin(end_situations)

O filtro de projetos na etapa de admissibilidade é a junção dos filtros anteriores

In [10]:
filter_admissibility = (filter_start_execution_bigger_than_currentdate &
                        filter_not_have_end_situation)

**Quantidade de projetos na fase de Admissibilidade**

In [11]:
df_projects_admissibility = df_projects[filter_admissibility]

display("Nº de projetos: {}".format(len(df_projects_admissibility)))

'Nº de projetos: 439'

## Identificando projetos na etapa de Execução/Captação

Segundo as informações passadas pelo pessoal do Ministério, a etapa de execução e captação ocorrem juntas, logo será apenas utilizado o nome "fase de execução"

### Preparando filtros

Para identificar se um projeto está na etapa de execução será verificado as seguintes informações

1. A data de inicio de execução de um projeto deve ser anterior a data atual.
2. A data de fim de execução de um projeto deve ser posterior a data atual.
3. O projeto não pode ter uma situação que indica que ele já foi finalizado.

**Filtro 1:** A data de inicio de execução de um projeto deve ser anterior a data atual.

In [12]:
filter_start_execution_before_than_currentdate = df_projects['DtInicioExecucao'] < current_date

**Filtro 2:** A data de fim de execução de um projeto deve ser posterior a data atual.

In [13]:
filter_end_execution_after_than_currentdate = df_projects['DtFimExecucao'] > current_date

**Filtro 3:** O projeto não pode ter uma situação que indica que ele já foi finalizado.

In [14]:
filter_not_have_end_situation = ~df_projects['Situacao'].isin(end_situations)

O filtro de projetos na etapa de execução é a junção dos filtros anteriores

In [15]:
filter_execution = (filter_start_execution_before_than_currentdate &
                    filter_end_execution_after_than_currentdate &
                    filter_not_have_end_situation)

**Quantidade de projetos na fase de Execução**

In [16]:
df_projects_execution = df_projects[filter_execution]

display("Nº de projetos: {}".format(len(df_projects_execution)))

# display(df_projects_execution.sort_values('DtFimExecucao'))

'Nº de projetos: 7490'

## Identificando projetos na etapa de Prestação de Contas

### Preparando filtros

Para identificar se um projeto está na etapa de prestação de contas será verificado as seguintes informações

1. A data de fim de execução de um projeto deve ser anterior a data atual.
2. O projeto não pode ter uma situação que indica que ele já foi finalizado.

**Filtro 1:** A data de fim de execução de um projeto deve ser anterior a data atual.

In [17]:
filter_end_execution_before_than_currentdate = df_projects['DtFimExecucao'] < current_date

**Filtro 3:** O projeto não pode ter uma situação que indica que ele já foi finalizado.

In [18]:
filter_not_have_end_situation = ~df_projects['Situacao'].isin(end_situations)

O filtro de projetos na etapa de execução é a junção dos filtros anteriores

In [19]:
filter_accountability = (filter_end_execution_before_than_currentdate &
                         filter_not_have_end_situation)

**Quantidade de projetos na fase de Prestação de Contas**

In [20]:
df_projects_accountability = df_projects[filter_accountability]

display("Nº de projetos: {}".format(len(df_projects_accountability)))

'Nº de projetos: 17091'

## Quantidade de projetos em todas as etapas

In [23]:
display("Admissibilidade: {}".format(len(df_projects_admissibility)))
display("Execução: {}".format(len(df_projects_execution)))
display("Prestação de contas: {}".format(len(df_projects_accountability)))
display("Total: {}".format(len(df_projects_accountability) + 
                           len(df_projects_accountability) + 
                           len(df_projects_accountability)))

'Admissibilidade: 439'

'Execução: 7490'

'Prestação de contas: 17091'

'Total: 51273'